### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
total_schools = len(school_data_complete["School ID"].unique())
total_students = len(school_data_complete["Student ID"].unique())
budget_per_school = school_data_complete["budget"].unique()
total_budget = budget_per_school.sum()
avg_math_score = school_data_complete["math_score"].mean()
avg_read_score = school_data_complete["reading_score"].mean()

students_pass_math_df = school_data_complete.loc[school_data_complete["math_score"] >= 70, :]
num_pass_math = students_pass_math_df["Student ID"].count()
pct_pass_math = num_pass_math / total_students * 100

students_pass_read_df = school_data_complete.loc[school_data_complete["reading_score"] >= 70, :]
num_pass_read = students_pass_read_df["Student ID"].count()
pct_pass_read = num_pass_read / total_students * 100

students_pass_math_read_df = school_data_complete.loc[(school_data_complete["reading_score"] >= 70) & 
                                                      (school_data_complete["math_score"] >= 70), :] 
num_pass_math_read = students_pass_math_read_df["Student ID"].count()
pct_pass_both = num_pass_math_read / total_students * 100

district_summary_df = pd.DataFrame({
                            "Total Schools":[total_schools],
                            "Total Students":[total_students],
                            "Total Budget":[total_budget],
                            "Average Math Score":[avg_math_score],
                            "Average Reading Score":[avg_read_score],
                            "% Passing Math":[pct_pass_math],
                            "% Passing Reading":[pct_pass_read],
                            "% Overall Passing":[pct_pass_both]
                                   })

district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)
district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].map("{:,.2f}".format)
district_summary_df["Average Reading Score"] = district_summary_df["Average Reading Score"].map("{:,.2f}".format)
district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].map("{:,.2f}%".format)
district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].map("{:,.2f}%".format)
district_summary_df["% Overall Passing"] = district_summary_df["% Overall Passing"].map("{:,.2f}%".format)

district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98%,85.81%,65.17%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [3]:
school_grouped_df = school_data_complete.groupby(["school_name"])

school_type = school_grouped_df["type"].min()
school_total_students = school_grouped_df["Student ID"].count()
unique_total_school_budget = school_grouped_df["budget"].min()
per_student_budget = unique_total_school_budget / school_total_students
school_avg_math_score = school_grouped_df["math_score"].mean()
school_avg_read_score = school_grouped_df["reading_score"].mean()

school_students_pass_math_df = school_data_complete.loc[school_data_complete["math_score"] >= 70, :]
grouped_pass_math_df = school_students_pass_math_df.groupby(["school_name"])
school_num_pass_math = grouped_pass_math_df["Student ID"].count()
school_pct_pass_math = school_num_pass_math / school_total_students * 100

school_students_pass_read_df = school_data_complete.loc[school_data_complete["reading_score"] >= 70, :]
grouped_pass_read_df = school_students_pass_read_df.groupby(["school_name"])
school_num_pass_read = grouped_pass_read_df["Student ID"].count()
school_pct_pass_read = school_num_pass_read / school_total_students * 100

school_students_pass_math_read_df = school_data_complete.loc[(school_data_complete["reading_score"] >= 70) & 
                                                      (school_data_complete["math_score"] >= 70), :] 
grouped_pass_both_df = school_students_pass_math_read_df.groupby(["school_name"])
num_pass_both = grouped_pass_both_df["Student ID"].count()
pct_pass_both = num_pass_both / school_total_students * 100

school_summary_df = pd.merge(school_type, school_total_students, on="school_name")
school_summary_df["Total School Budget"] = unique_total_school_budget
school_summary_df["Per Student Budget"] = per_student_budget
school_summary_df["Average Math Score"] = school_avg_math_score
school_summary_df["Average Reading Score"] = school_avg_read_score
school_summary_df["% Passing Math"] = school_pct_pass_math
school_summary_df["% Passing Reading"] = school_pct_pass_read
school_summary_df["% Overall Passing"] = pct_pass_both

school_summary_df = school_summary_df.rename(columns={"type":"School Type", "Student ID":"Total Students"})

school_summary_df["Total School Budget"] = school_summary_df["Total School Budget"].map("${:,.2f}".format)
school_summary_df["Per Student Budget"] = school_summary_df["Per Student Budget"].map("${:,.2f}".format)

school_summary_df.index.name = None
school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [4]:
top_performing_sort_df = school_summary_df.sort_values(["% Overall Passing"], ascending=False)
top_performing_sort_df.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [5]:
bottom_performing_sort_df = school_summary_df.sort_values(["% Overall Passing"])
bottom_performing_sort_df.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [6]:
ninth_grade = school_data_complete.loc[school_data_complete["grade"] == "9th", :]
tenth_grade = school_data_complete.loc[school_data_complete["grade"] == "10th", :]
eleventh_grade = school_data_complete.loc[school_data_complete["grade"] == "11th", :]
twelfth_grade = school_data_complete.loc[school_data_complete["grade"] == "12th", :]

ninth_grouped = ninth_grade.groupby(["school_name"])
tenth_grouped = tenth_grade.groupby(["school_name"])
eleventh_grouped = eleventh_grade.groupby(["school_name"])
twelfth_grouped = twelfth_grade.groupby(["school_name"])

ninth_math_avg = ninth_grouped["math_score"].mean()
tenth_math_avg = tenth_grouped["math_score"].mean()
eleventh_math_avg = eleventh_grouped["math_score"].mean()
twelfth_math_avg = twelfth_grouped["math_score"].mean()

math_by_grade_df = pd.merge(ninth_math_avg, tenth_math_avg, on="school_name")
math_by_grade_df["11th"] = eleventh_math_avg
math_by_grade_df["12th"] = twelfth_math_avg

math_by_grade_df = math_by_grade_df.rename(columns={
                                                    "math_score_x":"9th",
                                                    "math_score_y":"10th"
})

math_by_grade_df["9th"] = math_by_grade_df["9th"].map("{:,.2f}".format)
math_by_grade_df["10th"] = math_by_grade_df["10th"].map("{:,.2f}".format)
math_by_grade_df["11th"] = math_by_grade_df["11th"].map("{:,.2f}".format)
math_by_grade_df["12th"] = math_by_grade_df["12th"].map("{:,.2f}".format)

math_by_grade_df.index.name = None
math_by_grade_df

,9th,10th,11th,12th
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86
Pena High School,83.63,83.37,84.33,84.12


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [7]:
read_ninth_avg = ninth_grouped["reading_score"].mean()
read_tenth_avg = tenth_grouped["reading_score"].mean()
read_eleventh_avg = eleventh_grouped["reading_score"].mean()
read_twelfth_avg = twelfth_grouped["reading_score"].mean()

read_by_grade_df = pd.merge(read_ninth_avg, read_tenth_avg, on="school_name")
read_by_grade_df["11th"] = read_eleventh_avg
read_by_grade_df["12th"] = read_twelfth_avg

read_by_grade_df = read_by_grade_df.rename(columns={
                                                    "reading_score_x":"9th",
                                                    "reading_score_y":"10th"
})

read_by_grade_df["9th"] = read_by_grade_df["9th"].map("{:,.2f}".format)
read_by_grade_df["10th"] = read_by_grade_df["10th"].map("{:,.2f}".format)
read_by_grade_df["11th"] = read_by_grade_df["11th"].map("{:,.2f}".format)
read_by_grade_df["12th"] = read_by_grade_df["12th"].map("{:,.2f}".format)

read_by_grade_df.index.name = None
read_by_grade_df

,9th,10th,11th,12th
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23
Pena High School,83.81,83.61,84.34,84.59


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [8]:
bins = [0,585,629,645,675]
label_names = ["<$584","$585-629","$630-644","$645-675"]

school_spend_df = school_summary_df[school_summary_df.columns[1:]].replace('[\$,]', '', regex=True).astype(float)

school_spend_df['Per Student Budget'] = pd.to_numeric(school_spend_df['Per Student Budget'])

school_spend_df["Spending Ranges (Per Student)"] = pd.cut(school_spend_df["Per Student Budget"], 
                                                                   bins, labels=label_names, include_lowest=True)

spending_grouped_df = school_spend_df.groupby(["Spending Ranges (Per Student)"])

spending_math_avg = spending_grouped_df["Average Math Score"].mean()
spending_read_avg = spending_grouped_df["Average Reading Score"].mean()
spending_pct_pass_math = spending_grouped_df["% Passing Math"].mean()
spending_pct_pass_read = spending_grouped_df["% Passing Reading"].mean()
spending_pct_pass_both = spending_grouped_df["% Overall Passing"].mean()

scores_school_spend_df = pd.merge(spending_math_avg, spending_read_avg, on="Spending Ranges (Per Student)")
scores_school_spend_df = pd.merge(scores_school_spend_df,spending_pct_pass_math, on="Spending Ranges (Per Student)")
scores_school_spend_df = pd.merge(scores_school_spend_df,spending_pct_pass_read, on="Spending Ranges (Per Student)")
scores_school_spend_df = pd.merge(scores_school_spend_df,spending_pct_pass_both, on="Spending Ranges (Per Student)")

scores_school_spend_df["Average Math Score"] = scores_school_spend_df["Average Math Score"].map("{:,.2f}".format)
scores_school_spend_df["Average Reading Score"] = scores_school_spend_df["Average Reading Score"].map("{:,.2f}".format)
scores_school_spend_df["% Passing Math"] = scores_school_spend_df["% Passing Math"].map("{:,.2f}".format)
scores_school_spend_df["% Passing Reading"] = scores_school_spend_df["% Passing Reading"].map("{:,.2f}".format)
scores_school_spend_df["% Overall Passing"] = scores_school_spend_df["% Overall Passing"].map("{:,.2f}".format)

scores_school_spend_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$584,83.46,83.93,93.46,96.61,90.37
$585-629,81.90,83.16,87.13,92.72,81.42
$630-644,78.52,81.62,73.48,84.39,62.86
$645-675,77.00,81.03,66.16,81.13,53.53


## Scores by School Size

* Perform the same operations as above, based on school size.

In [9]:
bins_size = [0,1000,2000,5000]
label_names_size = ["Small (<1000)","Medium (1000-2000)","Large (2000-5000)"]

school_size_df = school_summary_df

school_size_df["School Size"] = pd.cut(school_spend_df["Total Students"], 
                                                                bins_size, labels=label_names_size, include_lowest=True)

school_size_df = school_size_df.groupby(["School Size"])

size_math_avg = school_size_df["Average Math Score"].mean()
size_read_avg = school_size_df["Average Reading Score"].mean()
size_pct_pass_math = school_size_df["% Passing Math"].mean()
size_pct_pass_read = school_size_df["% Passing Reading"].mean()
size_pct_pass_both = school_size_df["% Overall Passing"].mean()

size_spend_df = pd.merge(size_math_avg, size_read_avg, on="School Size")
size_spend_df = pd.merge(size_spend_df,size_pct_pass_math, on="School Size")
size_spend_df = pd.merge(size_spend_df,size_pct_pass_read, on="School Size")
size_spend_df = pd.merge(size_spend_df,size_pct_pass_both, on="School Size")

size_spend_df["Average Math Score"] = size_spend_df["Average Math Score"].map("{:,.2f}".format)
size_spend_df["Average Reading Score"] = size_spend_df["Average Reading Score"].map("{:,.2f}".format)
size_spend_df["% Passing Math"] = size_spend_df["% Passing Math"].map("{:,.2f}".format)
size_spend_df["% Passing Reading"] = size_spend_df["% Passing Reading"].map("{:,.2f}".format)
size_spend_df["% Overall Passing"] = size_spend_df["% Overall Passing"].map("{:,.2f}".format)

size_spend_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.82,83.93,93.55,96.10,89.88
Medium (1000-2000),83.37,83.86,93.60,96.79,90.62
Large (2000-5000),77.75,81.34,69.96,82.77,58.29


## Scores by School Type

* Perform the same operations as above, based on school type

In [10]:
school_type_df = school_summary_df

school_type_df = school_type_df.groupby(["School Type"])

type_math_avg = school_type_df["Average Math Score"].mean()
type_read_avg = school_type_df["Average Reading Score"].mean()
type_pct_pass_math = school_type_df["% Passing Math"].mean()
type_pct_pass_read = school_type_df["% Passing Reading"].mean()
type_pct_pass_both = school_type_df["% Overall Passing"].mean()

type_spend_df = pd.merge(type_math_avg, type_read_avg, on="School Type")
type_spend_df = pd.merge(type_spend_df,type_pct_pass_math, on="School Type")
type_spend_df = pd.merge(type_spend_df,type_pct_pass_read, on="School Type")
type_spend_df = pd.merge(type_spend_df,type_pct_pass_both, on="School Type")

type_spend_df["Average Math Score"] = type_spend_df["Average Math Score"].map("{:,.2f}".format)
type_spend_df["Average Reading Score"] = type_spend_df["Average Reading Score"].map("{:,.2f}".format)
type_spend_df["% Passing Math"] = type_spend_df["% Passing Math"].map("{:,.2f}".format)
type_spend_df["% Passing Reading"] = type_spend_df["% Passing Reading"].map("{:,.2f}".format)
type_spend_df["% Overall Passing"] = type_spend_df["% Overall Passing"].map("{:,.2f}".format)

type_spend_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.47,83.90,93.62,96.59,90.43
District,76.96,80.97,66.55,80.80,53.67
